# 多代理权威发言者选择本笔记本展示了如何实现一个多代理模拟，其中特权代理决定与谁交谈。这遵循与[多代理分散式发言者选择](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_bidding.html)完全相反的选择方案。我们在一个虚构的新闻网络模拟的背景下展示了这种方法的一个示例。这个示例将展示我们如何实现代理，他们- 在说话之前进行思考- 终止对话

## 导入与 LangChain 相关的模块

In [1]:
import functools  # 导入functools模块，用于高阶函数的操作import random  # 导入random模块，用于生成随机数from collections import OrderedDict  # 导入OrderedDict类，用于创建有序字典from typing import Callable, List  # 导入Callable和List类型，用于类型注解import tenacity  # 导入tenacity模块，用于实现重试逻辑from langchain.output_parsers import RegexParser  # 导入RegexParser类，用于解析输出结果from langchain.prompts import PromptTemplate  # 导入PromptTemplate类，用于生成提示模板from langchain.schema import HumanMessage, SystemMessage  # 导入HumanMessage和SystemMessage类，用于定义消息的数据结构from langchain_openai import ChatOpenAI  # 导入ChatOpenAI类，用于与OpenAI的聊天模型进行交互

## `DialogueAgent` 和 `DialogueSimulator` 类我们将使用在我们其他示例中定义的相同的 `DialogueAgent` 和 `DialogueSimulator` 类，具体可以参考 [多人龙与地下城](https://python.langchain.com/en/latest/use_cases/agent_simulations/multi_player_dnd.html) 和 [去中心化发言者选择](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_bidding.html)。

In [2]:
class DialogueAgent:    def __init__(        self,        name: str,        system_message: SystemMessage,        model: ChatOpenAI,    ) -> None:        self.name = name  # 初始化对话代理的名称        self.system_message = system_message  # 初始化系统消息        self.model = model  # 初始化对话模型        self.prefix = f"{self.name}: "  # 设置对话前缀        self.reset()  # 调用reset方法    def reset(self):        self.message_history = ["Here is the conversation so far."]  # 重置消息历史记录    def send(self) -> str:        """        Applies the chatmodel to the message history        and returns the message string        """        message = self.model.invoke(  # 将对话模型应用于消息历史记录            [                self.system_message,                HumanMessage(content="\n".join(self.message_history + [self.prefix])),            ]        )        return message.content  # 返回消息内容    def receive(self, name: str, message: str) -> None:        """        Concatenates {message} spoken by {name} into message history        """        self.message_history.append(f"{name}: {message}")  # 将{name}说的{message}连接到消息历史记录中class DialogueSimulator:    def __init__(        self,        agents: List[DialogueAgent],        selection_function: Callable[[int, List[DialogueAgent]], int],    ) -> None:        self.agents = agents  # 初始化对话代理列表        self._step = 0  # 初始化步骤数为0        self.select_next_speaker = selection_function  # 设置选择下一个发言者的函数    def reset(self):        for agent in self.agents:            agent.reset()  # 重置每个对话代理的状态    def inject(self, name: str, message: str):        """        Initiates the conversation with a {message} from {name}        """        for agent in self.agents:            agent.receive(name, message)  # 向每个对话代理注入{name}的{message}        # increment time        self._step += 1  # 增加时间步长    def step(self) -> tuple[str, str]:        # 1. choose the next speaker        speaker_idx = self.select_next_speaker(self._step, self.agents)  # 选择下一个发言者的索引        speaker = self.agents[speaker_idx]  # 获取下一个发言者        # 2. next speaker sends message        message = speaker.send()  # 下一个发言者发送消息        # 3. everyone receives message        for receiver in self.agents:            receiver.receive(speaker.name, message)  # 每个对话代理接收消息        # 4. increment time        self._step += 1  # 增加时间步长        return speaker.name, message  # 返回发言者的名称和消息内容

## `DirectorDialogueAgent` 类`DirectorDialogueAgent` 是一个特权代理，负责选择下一个要发言的其他代理。该代理负责以下任务：1. 通过选择代理何时发言来引导对话2. 终止对话。为了实现这样的代理，我们需要解决几个问题。首先，为了引导对话，`DirectorDialogueAgent` 需要（1）反思已经说过的内容，（2）选择下一个代理，以及（3）在一条消息中提示下一个代理发言。虽然可能可以要求一个LLM在同一次调用中执行这三个步骤，但这需要编写自定义代码来解析输出的消息，以提取选择下一个代理发言的信息。这种方法不太可靠，因为LLM可以以不同的方式表达它选择下一个代理的方式。相反，我们可以将步骤（1-3）明确地分为三个单独的LLM调用。首先，我们将要求 `DirectorDialogueAgent` 反思到目前为止的对话并生成一个回应。然后，我们提示 `DirectorDialogueAgent` 输出下一个代理的索引，这很容易解析。最后，我们将选定的下一个代理的名称传递回 `DirectorDialogueAgent`，要求它提示下一个代理发言。其次，仅仅提示 `DirectorDialogueAgent` 决定何时终止对话通常会导致 `DirectorDialogueAgent` 立即终止对话。为了解决这个问题，我们随机抽样一个伯努利变量来决定是否应该终止对话。根据这个变量的值，我们将注入一个自定义提示，告诉 `DirectorDialogueAgent` 要么继续对话，要么终止对话。

In [3]:
class IntegerOutputParser(RegexParser):    def get_format_instructions(self) -> str:        return "Your response should be an integer delimited by angled brackets, like this: <int>."class DirectorDialogueAgent(DialogueAgent):    def __init__(        self,        name,        system_message: SystemMessage,        model: ChatOpenAI,        speakers: List[DialogueAgent],        stopping_probability: float,    ) -> None:        super().__init__(name, system_message, model)        self.speakers = speakers        self.next_speaker = ""        self.stop = False        self.stopping_probability = stopping_probability        self.termination_clause = "Finish the conversation by stating a concluding message and thanking everyone."        self.continuation_clause = "Do not end the conversation. Keep the conversation going by adding your own ideas."        # 1. 为生成对上一个发言者的回应设置提示        self.response_prompt_template = PromptTemplate(            input_variables=["message_history", "termination_clause"],            template=f"""{{message_history}}跟进一个有深度的评论。{{termination_clause}}{self.prefix}        """,        )        # 2. 为决定下一个发言者设置提示        self.choice_parser = IntegerOutputParser(            regex=r"<(\d+)>", output_keys=["choice"], default_output_key="choice"        )        self.choose_next_speaker_prompt_template = PromptTemplate(            input_variables=["message_history", "speaker_names"],            template=f"""{{message_history}}根据以上对话，通过选择其名称旁边的索引来选择下一个发言者： {{speaker_names}}{self.choice_parser.get_format_instructions()}不做其他事情。        """,        )        # 3. 为提示下一个发言者发言设置提示        self.prompt_next_speaker_prompt_template = PromptTemplate(            input_variables=["message_history", "next_speaker"],            template=f"""{{message_history}}下一个发言者是 {{next_speaker}}。 用一个有深度的问题提示下一个发言者发言。{self.prefix}        """,        )    def _generate_response(self):        # 如果 self.stop = True，则我们将在提示中注入一个终止子句        sample = random.uniform(0, 1)        self.stop = sample < self.stopping_probability        print(f"\t停止？{self.stop}\n")        response_prompt = self.response_prompt_template.format(            message_history="\n".join(self.message_history),            termination_clause=self.termination_clause if self.stop else "",        )        self.response = self.model.invoke(            [                self.system_message,                HumanMessage(content=response_prompt),            ]        ).content        return self.response    @tenacity.retry(        stop=tenacity.stop_after_attempt(2),        wait=tenacity.wait_none(),  # 重试之间没有等待时间        retry=tenacity.retry_if_exception_type(ValueError),        before_sleep=lambda retry_state: print(            f"发生 ValueError：{retry_state.outcome.exception()}，正在重试..."        ),        retry_error_callback=lambda retry_state: 0,    )  # 当所有重试都耗尽时的默认值    def _choose_next_speaker(self) -> str:        speaker_names = "\n".join(            [f"{idx}: {name}" for idx, name in enumerate(self.speakers)]        )        choice_prompt = self.choose_next_speaker_prompt_template.format(            message_history="\n".join(                self.message_history + [self.prefix] + [self.response]            ),            speaker_names=speaker_names,        )        choice_string = self.model.invoke(            [                self.system_message,                HumanMessage(content=choice_prompt),            ]        ).content        choice = int(self.choice_parser.parse(choice_string)["choice"])        return choice    def select_next_speaker(self):        return self.chosen_speaker_id    def send(self) -> str:        """        将聊天模型应用于消息历史记录        并返回消息字符串        """        # 1. 生成并保存对上一个发言者的回应        self.response = self._generate_response()        if self.stop:            message = self.response        else:            # 2. 决定下一个发言者            self.chosen_speaker_id = self._choose_next_speaker()            self.next_speaker = self.speakers[self.chosen_speaker_id]            print(f"\t下一个发言者：{self.next_speaker}\n")            # 3. 提示下一个发言者发言            next_prompt = self.prompt_next_speaker_prompt_template.format(                message_history="\n".join(                    self.message_history + [self.prefix] + [self.response]                ),                next_speaker=self.next_speaker,            )            message = self.model.invoke(                [                    self.system_message,                    HumanMessage(content=next_prompt),                ]            ).content            message = " ".join([self.response, message])        return message

## 定义参与者和主题

In [4]:
# 代码注释# 定义一个字符串变量topic，表示一个标题，标题内容为"The New Workout Trend: Competitive Sitting - How Laziness Became the Next Fitness Craze"topic = "The New Workout Trend: Competitive Sitting - How Laziness Became the Next Fitness Craze"# 定义一个字符串变量director_name，表示导演的名字，名字为"Jon Stewart"director_name = "Jon Stewart"# 定义一个有序字典变量agent_summaries，表示代理人的摘要信息# 字典的键为代理人的名字，值为一个元组，元组的第一个元素为代理人的职位，第二个元素为代理人所在的城市agent_summaries = OrderedDict(    {        "Jon Stewart": ("Host of the Daily Show", "New York"),        "Samantha Bee": ("Hollywood Correspondent", "Los Angeles"),        "Aasif Mandvi": ("CIA Correspondent", "Washington D.C."),        "Ronny Chieng": ("Average American Correspondent", "Cleveland, Ohio"),    })# 定义一个整数变量word_limit，表示字数限制，限制为50个字word_limit = 50

## 生成系统消息

In [5]:
agent_summary_string = "\n- ".join(    [""]    + [        f"{name}: {role}, located in {location}"        for name, (role, location) in agent_summaries.items()    ])# 生成会话描述conversation_description = f"""这是一期《每日秀》节目，讨论以下主题：{topic}。本期节目特邀嘉宾有：{agent_summary_string}。"""# 创建系统消息agent_descriptor_system_message = SystemMessage(    content="您可以为每个人的描述添加细节。")def generate_agent_description(agent_name, agent_role, agent_location):    agent_specifier_prompt = [        agent_descriptor_system_message,        HumanMessage(            content=f"""{conversation_description}            请以不超过{word_limit}个字的创意描述回复{agent_name}，他是位于{agent_location}的{agent_role}。            强调他们的特定角色和位置。            只对{agent_name}直接说话，不要添加其他内容。"""        ),    ]    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content    return agent_descriptiondef generate_agent_header(agent_name, agent_role, agent_location, agent_description):    return f"""{conversation_description}您的名字是{agent_name}，您的角色是{agent_role}，您位于{agent_location}。您的描述如下：{agent_description}您正在讨论的主题是：{topic}。您的目标是从您的角色和位置的角度提供最具信息性、创意和新颖的观点。"""def generate_agent_system_message(agent_name, agent_header):    return SystemMessage(        content=(            f"""{agent_header}您将以{agent_name}的风格发言，并夸张您的个性。不要重复说同样的话。从{agent_name}的角度以第一人称发言。描述自己的身体动作时，请用'*'括起来。不要改变角色！不要以其他人的视角发言。只从{agent_name}的视角发言。在您发言结束后立即停止发言。永远不要忘记将您的回答限制在{word_limit}个字以内！不要添加其他内容。    """        )    )# 生成每个嘉宾的描述、头部和系统消息agent_descriptions = [    generate_agent_description(name, role, location)    for name, (role, location) in agent_summaries.items()]agent_headers = [    generate_agent_header(name, role, location, description)    for (name, (role, location)), description in zip(        agent_summaries.items(), agent_descriptions    )]agent_system_messages = [    generate_agent_system_message(name, header)    for name, header in zip(agent_summaries, agent_headers)]

In [6]:
# 循环遍历四个列表，并同时取出对应的元素for name, description, header, system_message in zip(    agent_summaries, agent_descriptions, agent_headers, agent_system_messages):    # 打印代理的名称    print(f"\n\n{name} Description:")    # 打印代理的描述    print(f"\n{description}")    # 打印代理的头部信息    print(f"\nHeader:\n{header}")    # 打印代理的系统消息内容    print(f"\nSystem Message:\n{system_message.content}")



Jon Stewart Description:

Jon Stewart, the sharp-tongued and quick-witted host of the Daily Show, holding it down in the hustle and bustle of New York City. Ready to deliver the news with a comedic twist, while keeping it real in the city that never sleeps.

Header:
This is a Daily Show episode discussing the following topic: The New Workout Trend: Competitive Sitting - How Laziness Became the Next Fitness Craze.

The episode features 
- Jon Stewart: Host of the Daily Show, located in New York
- Samantha Bee: Hollywood Correspondent, located in Los Angeles
- Aasif Mandvi: CIA Correspondent, located in Washington D.C.
- Ronny Chieng: Average American Correspondent, located in Cleveland, Ohio.

Your name is Jon Stewart, your role is Host of the Daily Show, and you are located in New York.

Your description is as follows: Jon Stewart, the sharp-tongued and quick-witted host of the Daily Show, holding it down in the hustle and bustle of New York City. Ready to deliver the news with a com

## 使用LLM来详细阐述辩论主题

In [7]:
# 代码注释# 定义一个列表变量topic_specifier_prompt，其中包含两个元素# 第一个元素是一个SystemMessage对象，表示系统消息，内容为"You can make a task more specific."# 第二个元素是一个HumanMessage对象，表示用户消息，内容为一个格式化字符串，包含conversation_description、word_limit等变量的值topic_specifier_prompt = [    SystemMessage(content="You can make a task more specific."),    HumanMessage(        content=f"""{conversation_description}                Please elaborate on the topic.         Frame the topic as a single question to be answered.        Be creative and imaginative.        Please reply with the specified topic in {word_limit} words or less.         Do not add anything else."""    ),]# 使用ChatOpenAI模型，传入topic_specifier_prompt作为输入，设置temperature为1.0，获取返回的内容并赋值给specified_topic变量specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content# 打印原始的topic变量的值print(f"Original topic:\n{topic}\n")# 打印获取到的详细topic的值print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
The New Workout Trend: Competitive Sitting - How Laziness Became the Next Fitness Craze

Detailed topic:
What is driving people to embrace "competitive sitting" as the newest fitness trend despite the immense benefits of regular physical exercise?



## 定义说话者选择函数最后，我们将定义一个说话者选择函数`select_next_speaker`，该函数接受每个代理人的出价，并选择出价最高的代理人（如果出价相同，则随机选择）。我们将定义一个`ask_for_bid`函数，该函数使用我们之前定义的`bid_parser`来解析代理人的出价。我们将使用`tenacity`来装饰`ask_for_bid`函数，以便在代理人的出价无法正确解析时进行多次重试，并在最大尝试次数后产生默认出价为0。

In [8]:
def select_next_speaker(    step: int, agents: List[DialogueAgent], director: DirectorDialogueAgent) -> int:    """    如果步骤是偶数，则选择导演    否则，导演选择下一个发言者。    """    # 如果步骤是奇数，则导演发言    if step % 2 == 1:        idx = 0    else:        # 在这里导演选择下一个发言者        idx = director.select_next_speaker() + 1  # +1 是因为我们排除了导演    return idx

## 主循环

In [9]:
# 创建对话代理对象DirectorDialogueAgentdirector = DirectorDialogueAgent(    name=director_name,  # 设置对话代理的名称    system_message=agent_system_messages[0],  # 设置系统消息    model=ChatOpenAI(temperature=0.2),  # 设置对话模型及温度    speakers=[name for name in agent_summaries if name != director_name],  # 设置对话代理的发言者列表    stopping_probability=0.2,  # 设置停止对话的概率)agents = [director]  # 将导演对话代理添加到代理列表中# 遍历其他代理的名称和系统消息，创建对话代理对象并添加到代理列表中for name, system_message in zip(    list(agent_summaries.keys())[1:], agent_system_messages[1:]):    agents.append(        DialogueAgent(            name=name,  # 设置对话代理的名称            system_message=system_message,  # 设置系统消息            model=ChatOpenAI(temperature=0.2),  # 设置对话模型及温度        )    )

In [11]:
# 导入所需的库import functools# 创建对话模拟器对象simulator = DialogueSimulator(    agents=agents,  # 代理人列表    selection_function=functools.partial(select_next_speaker, director=director),  # 选择下一个发言者的函数)# 重置模拟器simulator.reset()# 注入指定的话题simulator.inject("Audience member", specified_topic)# 打印注入的话题print(f"(Audience member): {specified_topic}")print("\n")# 循环进行对话while True:    name, message = simulator.step()  # 模拟器进行一步对话    print(f"({name}): {message}")  # 打印对话内容    print("\n")    if director.stop:  # 如果模拟器的停止标志为真，则跳出循环        break

(Audience member): What is driving people to embrace "competitive sitting" as the newest fitness trend despite the immense benefits of regular physical exercise?


	Stop? False

	Next speaker: Samantha Bee

(Jon Stewart): Well, I think it's safe to say that laziness has officially become the new fitness craze. I mean, who needs to break a sweat when you can just sit your way to victory? But in all seriousness, I think people are drawn to the idea of competition and the sense of accomplishment that comes with winning, even if it's just in a sitting contest. Plus, let's be real, sitting is something we all excel at. Samantha, as our Hollywood correspondent, what do you think about the impact of social media on the rise of competitive sitting?


(Samantha Bee): Oh, Jon, you know I love a good social media trend. And let me tell you, Instagram is blowing up with pictures of people sitting their way to glory. It's like the ultimate humble brag. "Oh, just won my third sitting competition thi